# Human Annotation - Dataset to Qualtrics Script
GOAL: Create a .txt file that can be imported easily into Qualtrics to autogenerate a form following this documentation (
https://www.qualtrics.com/support/survey-platform/survey-module/survey-tools/import-and-export-surveys/#PreparingASimpleFormatTXTFile)

Sample .txt structure:

[[Block: Language Proficiency]]  
What languages are you fluent/comfortable in translating and interpreting sentences in?  
[[Question:MC:MultipleAnswer]]  
[[Choices]]  
A. English  
B. Simplified Chinese  
C. Traditional Chinese  
D. Hindi  
E. Spanish  
F. Arabic  

[[Block: Questions 1-10]]  
MC1. Query: XYZ. Passage: XYZ.  
[[Question:MC:SingleAnswer]]  
[[Choices]]  
A. Relevant  
B. Not relevant  

[[PageBreak]]  

[[Block: Questions 11-20]]  
MC11. Query: XYZ. Passage: XYZ.  
[[Question:MC:SingleAnswer]]  
[[Choices]]  
A. Relevant  
B. Not relevant  



### Detailed Questions for Form

We are collecting human annotations of query and passage pairs over different languages. We need your help in evaluating the relevance of each passage to its corresponding query. In this form, we will ask you to evaluate the relevance of 50 query-passage pairs in [LANGUAGE]. For each question, select either answer choice A or B.

In [ ]:
# import dataset
from datasets import load_dataset

ds_control = load_dataset(
    "borderlines/bordirlines", "control", split="openai.en", n_hits=10
)

In [ ]:
# get all engish query/doc pairs from dataset

In [ ]:
# string manip to put it in the form question structure

In [ ]:
# construct txt file form

In [ ]:
# export txt file